## Part 1 

In [1]:
# 1. first

from bs4 import BeautifulSoup
dokumen = '''
<html>
<head>
    <title>Tutorial BeautifulSoup</title>
</head>

<body>
    <p class="judul">Judul Dokumen</p>

    <p class="paragraf">Ini adalah contoh paragraf</p>

    <a href="https://ngodingdata.com" class="url">Ngodingdata</a>
</body>

</html>
'''

In [2]:
html_soup = BeautifulSoup(dokumen, 'html.parser')

print(html_soup)


<html>
<head>
<title>Tutorial BeautifulSoup</title>
</head>
<body>
<p class="judul">Judul Dokumen</p>
<p class="paragraf">Ini adalah contoh paragraf</p>
<a class="url" href="https://ngodingdata.com">Ngodingdata</a>
</body>
</html>



In [3]:
# fungsi find()
judul = html_soup.find('p')
print(judul)

<p class="judul">Judul Dokumen</p>


In [4]:
judul = html_soup.find('p', class_='judul')
paragraf = html_soup.find('p', class_='paragraf')
print(judul)
print(paragraf)

<p class="judul">Judul Dokumen</p>
<p class="paragraf">Ini adalah contoh paragraf</p>


In [5]:
# get only text 
judul_saja = html_soup.find('p', class_='judul').text
print(judul_saja)

Judul Dokumen


In [6]:
# find all 
all_paragraf = html_soup.find_all('p')
print(all_paragraf)

[<p class="judul">Judul Dokumen</p>, <p class="paragraf">Ini adalah contoh paragraf</p>]


## Part 2 - ETL

### Extract

In [7]:
import requests
page = requests.get("http://quotes.toscrape.com/")

print(page.status_code)
print(page.content)


200
b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\xe2\

In [8]:
from bs4 import BeautifulSoup
import requests


page = requests.get("http://quotes.toscrape.com/")
soup = BeautifulSoup(page.content, 'html.parser')

quote = soup.find('span', class_='text')

print(quote)

<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>


### Transform

In [9]:
# quote 
quote = soup.find('span', class_='text').text
print(quote)

author = soup.find('small', class_='author')
print(author)

tags = [tag.text for tag in soup.find('div', class_='tags').find_all('a', class_='tag')]
print(tags)


“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
<small class="author" itemprop="author">Albert Einstein</small>
['change', 'deep-thoughts', 'thinking', 'world']


### Find ALL

In [11]:
quotes = soup.find_all('div', class_='quote')
print(quotes)

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>, <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" itemprop="author">J.K.

In [14]:
for q in quotes:
    quote = q.find('span', class_='text').text
    author = q.find('small', class_='author').text
    tags = [tag.text for tag in q.find('div', class_='tags').find_all('a', class_='tag')]

    print(quote)
    print(author)
    print(tags)
    print()

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
['change', 'deep-thoughts', 'thinking', 'world']

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
['abilities', 'choices']

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
['inspirational', 'life', 'live', 'miracle', 'miracles']

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
['aliteracy', 'books', 'classic', 'humor']

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
['be-yourself', 'inspirational']

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
['adulthood', 'success', 'value']

“It is better to be hated for what you are tha

### Load to Database

In [16]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="db_quotes"
)

cursor = mydb.cursor()



In [18]:
for q in quotes:
    quote = q.find('span', class_='text').text
    author = q.find('small', class_='author').text
    tags = [tag.text for tag in q.find('div', class_='tags').find_all('a', class_='tag')]

    # print(quote)
    # print(author)
    # print(tags)
    # print()

    query = "INSERT INTO quotes (quote, author, tags) VALUES (%s, %s, %s)"
    value = (str(quote), str(author), str(tags))

    cursor.execute(query, value)
    mydb.commit()

    print("Data baru berhasil ditambahkan")

Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
Data baru berhasil ditambahkan
